In [1]:
import numpy as np
import sys, csv
import ipywidgets as widgets
from IPython.display import display
from os import system
from scipy.stats import norm
import scipy.integrate
from scipy.special import gamma, factorial
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astroquery.gaia import Gaia
import math
from zero_point import zpt
zpt.load_tables()

from astropy import units as u
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord
from astropy_healpix import HEALPix

import rpy2
from rpy2.robjects.packages import importr, data
import rpy2.robjects as robjects
from rpy2.robjects import numpy2ri
numpy2ri.activate()


In [2]:
r_source = robjects.r['source']
#r_source('./est_dist.R')
r_source('./Rfiles/Test.r')
r_photogeo_samples  = robjects.globalenv['distance.photgeo.samples']
r_ud_distpost5_photogeo  = robjects.globalenv['photgeo.dist.posterior']
r_photogeo_dist_prior = robjects.globalenv['photgeo.dist.prior']

R[write to console]: data.table 1.13.6 using 12 threads (see ?getDTthreads).  Latest news: r-datatable.com

R[write to console]: Lade nötiges Paket: bit

R[write to console]: 
Attache Paket: ‘bit’


R[write to console]: The following object is masked from ‘package:data.table’:

    setattr


R[write to console]: The following object is masked from ‘package:base’:

    xor


R[write to console]: Attaching package bit64

R[write to console]: package:bit64 (c) 2011-2017 Jens Oehlschlaegel

R[write to console]: creators: integer64 runif64 seq :

R[write to console]: coercion: as.integer64 as.vector as.logical as.integer as.double as.character as.bitstring

R[write to console]: logical operator: ! & | xor != == < <= >= >

R[write to console]: arithmetic operator: + - * / %/% %% ^

R[write to console]: math: sign abs sqrt log log2 log10

R[write to console]: math: floor ceiling trunc round

R[write to console]: querying: is.integer64 is.vector [is.atomic} [length] format print str

R[write t

In [3]:
# functions

def lim(r,rlo,rhi):
    return 1*np.bitwise_and(r>=rlo,r<=rhi) 
    
# Normalized Gaussian likelihood in w

def d_like(w,r,wsd):
    return norm.pdf(w,loc=1/r,scale=wsd)

# Unnormalized posterior in distance using 
# exponentially decreasing space density prior (with length scale rlen)

def ud_distpost3(r,w,wsd,rlen):
    
    return d_like(w=w,r=r,wsd=wsd)*lim(r=r,rlo=0,rhi=np.inf)*np.exp(-r/rlen)*r**2

#Unnormalized Posterior using GGD as prior
def prior4(r,rlen,alpha,beta):
    r = np.where(r > 0,r,0)
    return 1/gamma((beta+1)/alpha)*alpha/(rlen**(beta+1))*r**beta*np.exp(-((r/rlen)**alpha))

def ud_distpost4(r,w,wsd,rlen,alpha,beta):
    return d_like(w=w,r=r,wsd=wsd)*lim(r=r,rlo=0,rhi=np.inf)*prior4(r=r,rlen=rlen,alpha=alpha,beta=beta)

def metrop(func,thetaInit,Nburnin,Nsamp,sampleCov,verbose,demo=False,**kwargs):
    
    if not np.isscalar(thetaInit): 
        Ntheta = len(thetaInit)
    else:
        Ntheta = 1
        
    thetaCur = thetaInit
    funcCur = func(thetaInit,**kwargs)
    funcSamp = np.empty((Nsamp,2+Ntheta))
    funcSamp[:] = np.nan
    
    nAccept = 0
    acceptRate = 0
    
    if(demo):
        thetaPropAll = np.empty((Nsamp,Ntheta))
        thetaPropAll[:] = np.nan
    
    for n in np.arange(1,Nburnin+Nsamp+1):
        
        if np.isscalar(sampleCov):
            thetaProp = np.random.normal(loc=thetaCur, scale=np.sqrt(sampleCov), size=1)
        else:
            thetaProp = np.random.multivariate_normal(mean=thetaCur, cov=sampleCov, size = 1)
            
        funcProp = func(thetaProp,**kwargs)
        logMR = np.sum(funcProp) - np.sum(funcCur) 
        
        if logMR >= 0 or logMR > np.log10(np.random.uniform(low=0, high=1, size=1)):
            thetaCur = thetaProp
            funcCur = funcProp
            nAccept = nAccept + 1
            acceptRate = nAccept/n
        
        if n > Nburnin:
            funcSamp[n-Nburnin-1,0:2] = funcCur
            funcSamp[n-Nburnin-1,2:(2+Ntheta)] = thetaCur
            if(demo):
                thetaPropAll[n-Nburnin-1,0:Ntheta] = thetaProp
    
    if(demo):
        return {'funcSamp':funcSamp,'thetaPropAll':thetaPropAll}
    else:
        return funcSamp


def func_post3(r,w,wsd,rlen):
    return np.array([np.log10(ud_distpost3(r=r,w=w,wsd=wsd,rlen=rlen)),0],dtype=object)

def quantile_distpost3(w,wsd,rlen,rInit,rStep,Nburnin,Nsamp,probs):
    if np.any(np.array([w,wsd,rlen,rInit,probs],dtype=object))==np.nan:
        return {'code':0,'val':np.nan,'message':'some inputs NA'},np.nan
    if w == np.inf:
        return {'code':0,'val':np.nan,'message':'parallax not finite'},np.nan
    if not (wsd > 0 and wsd != np.inf):
        return {'code':0,'val':np.nan,'message':'parallax uncertainty not (finite and positive)'},np.nan
    if not (rlen > 0 and rlen != np.inf):
        return {'code':0,'val':np.nan,'message':'rlen not (finite and positive)'},np.nan
    if not (rInit > 0 and rInit != np.inf):
        return {'code':0,'val':np.nan,'message':'rInit not (finite and positive)'},np.nan
    if not (rStep > 0 and rStep != np.inf):
        return {'code':0,'val':np.nan,'message':'rStep not (finite and positive)'},np.nan
    if not (Nsamp > 0 and Nsamp != np.inf):
        return {'code':0,'val':np.nan,'message':'Nsamp not (finite and positive)'},np.nan
    if not (Nburnin > 0 and Nburnin != np.inf):
        return {'code':0,'val':np.nan,'message':'Nburnin not (finite and positive)'},np.nan
    if Nburnin >= Nsamp:
        return {'code':0,'val':np.nan,'message':'Nburnin >= Nsamp'},np.nan
    if np.any(probs<0) or np.any(probs>1):
        return {'code':0,'val':np.nan,'message':'probs not in range 0-1'},np.nan
   
    if ud_distpost3(r=rInit,w=w,wsd=wsd,rlen=rlen) <= 0:
        return {'code':0,'val':np.nan,'message':'metrop fails as posterior=zero at rInit'},np.nan
    
    samp = metrop(func=func_post3,thetaInit = rInit,Nburnin=Nburnin,Nsamp= Nsamp,verbose=np.inf,sampleCov=rStep**2,w=w,wsd=wsd,rlen=rlen)
    samp = np.array([i[2] for i in samp])
    
    return {'code':1,'val': np.quantile(samp,probs),'message':np.nan}, samp

def func_post4(r,w,wsd,rlen,alpha,beta): 
    return np.array([np.log10(ud_distpost4(r=r,w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta)),0],dtype=object)

        
def quantile_distpost4(w,wsd,rlen,alpha,beta,rInit,rStep,Nburnin,Nsamp,probs):
    
    if not (alpha > 0 and alpha != np.inf):
        return {'code':0,'val':np.nan,'message':'alpha not (finite and positive)'},np.nan
    if not (beta > 0 and beta != np.inf):
        return {'code':0,'val':np.nan,'message':'beta not (finite and positive)'},np.nan
    if np.any(np.array([w,wsd,rlen,alpha,beta,rInit,probs],dtype=object))==np.nan:
        return {'code':0,'val':np.nan,'message':'some inputs NA'},np.nan
    if w == np.inf:
        return {'code':0,'val':np.nan,'message':'parallax not finite'},np.nan
    if not (wsd > 0 and wsd != np.inf):
        return {'code':0,'val':np.nan,'message':'parallax uncertainty not (finite and positive)'},np.nan
    if not (rlen > 0 and rlen != np.inf):
        return {'code':0,'val':np.nan,'message':'rlen not (finite and positive)'},np.nan
    if not (rInit > 0 and rInit != np.inf):
        return {'code':0,'val':np.nan,'message':'rInit not (finite and positive)'},np.nan
    if not (rStep > 0 and rStep != np.inf):
        return {'code':0,'val':np.nan,'message':'rStep not (finite and positive)'},np.nan
    if not (Nsamp > 0 and Nsamp != np.inf):
        return {'code':0,'val':np.nan,'message':'Nsamp not (finite and positive)'},np.nan
    if not (Nburnin > 0 and Nburnin != np.inf):
        return {'code':0,'val':np.nan,'message':'Nburnin not (finite and positive)'},np.nan
    if Nburnin >= Nsamp:
        return {'code':0,'val':np.nan,'message':'Nburnin >= Nsamp'},np.nan
    if np.any(probs<0) or np.any(probs>1):
        return {'code':0,'val':np.nan,'message':'probs not in range 0-1'},np.nan   
    if ud_distpost4(r=rInit,w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta) <= 0.:
        return {'code':0,'val':np.nan,'message':'metrop fails as posterior=zero at rInit'},np.nan
    
    samp = metrop(func=func_post4,thetaInit = rInit,Nburnin=Nburnin,Nsamp=Nsamp,verbose=np.inf,sampleCov=rStep**2,w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta)
    samp = np.array([i[2] for i in samp])
    
    return {'code':1,'val': np.quantile(samp,probs),'message':np.nan},samp
    
def mode_post3(w,wsd,rlen,retall = False):
    # special cases:
    # w<=0 works normally, except for w=-Inf
    if np.isnan(np.array([w,wsd,rlen])).any(): 
        return np.nan
    if w == -np.inf:
        return np.inf
    if w == np.inf: 
        return 0
    if wsd == 0: 
        return 1/w
    if wsd == np.inf:
        return 2*rlen
    if wsd<0:
        return np.nan
    if rlen <=0:
        return np.nan
    
    coeff = [1/rlen,-2,w/wsd**2,-1/wsd**2] 
    roots = np.roots(coeff)    
    
    r = sum(np.isreal(roots)) # gives number of real modes
    
    if r == 0:
        rMode = np.nan
    if r == 1:
        rMode = np.extract(np.isreal(roots),roots)
    if r == 2: 
        rMode = np.nan
    if r == 3: 
        rMode = np.extract(np.isreal(roots),roots)
        if w>0:
            rMode = min(roots)
        else:
            rMode = roots[roots>0]
    
    if retall is True:
        return(roots)
    else:
        return(rMode.real)

    
def quantile_distpost5(w,wsd,hp,phot_g_mean_mag,bp_rp,pseudocolour,rInit,Nsamp,Nburnin,probs):
    
    if w == np.inf:
        return {'code':0,'val':np.nan,'message':'parallax not finite'},np.nan
    if not (wsd > 0 and wsd != np.inf):
        return {'code':0,'val':np.nan,'message':'parallax uncertainty not (finite and positive)'},np.nan
    if not (rInit > 0 and rInit != np.inf):
        return {'code':0,'val':np.nan,'message':'rInit not (finite and positive)'},np.nan
    if not (Nsamp > 0 and Nsamp != np.inf):
        return {'code':0,'val':np.nan,'message':'Nsamp not (finite and positive)'},np.nan
    if not (Nburnin > 0 and Nburnin != np.inf):
        return {'code':0,'val':np.nan,'message':'Nburnin not (finite and positive)'},np.nan
    if Nburnin >= Nsamp:
        return {'code':0,'val':np.nan,'message':'Nburnin >= Nsamp'},np.nan
    if np.any(probs<0) or np.any(probs>1):
        return {'code':0,'val':np.nan,'message':'probs not in range 0-1'},np.nan   
    if phot_g_mean_mag == np.inf:
        return {'code':0,'val':np.nan,'message':'G not finite'},np.nan  
    if bp_rp == np.inf:
        return {'code':0,'val':np.nan,'message':'colours not finite'},np.nan
    if hp <= 0 or hp >= 99:
        return {'code':0,'val':np.nan,'message':'HEALpixel out of range'},np.nan
    if w== np.nan or wsd == np.nan or hp == np.nan or rInit == np.nan or probs.any() == np.nan or phot_g_mean_mag == np.nan or bp_rp == np.nan:
            return {'code':0,'val':np.nan,'message':'some inputs NA'},np.nan
    
    samp0 = r_photogeo_samples(parallax=w,parallax_error=wsd,p=hp,phot_g_mean_mag=phot_g_mean_mag,bp_rp=bp_rp,pseudocolour = pseudocolour,rEDSDModeGeo=rInit,Nsamp=Nsamp,Nburnin=Nburnin)
    samp = samp0[0]
    flag = samp0[1]
    
    return {'code':1,'val': np.quantile(samp,probs),'message':np.nan,},samp,flag    

In [4]:
def f(custom,source_id,filename_in,filename_out,model,w,wsd,rlen,alpha,beta,metrop_start,metrop_step,metrop_Nsamp,metrop_Nburnin):
    np.seterr(divide = 'ignore') 
    plt.style.use('ggplot')
    
    # general setups:
    
    HDIprob = 0.6827
    
    #Range for normalizing Posterior
    rlo = 0
    rhi = 1e5
    #Plotting range: lower limit
    rplotlo = 0
    
    with open('prior_summary.csv', newline='') as prior_summary:
        reader_prior_summary = csv.reader(prior_summary)
        rows_prior_summary = []
        for row in reader_prior_summary:
            rows_prior_summary.append(row)
            
# Input CSVfile (name, parallax, parallax_error, ra, dec)-----------------------------------------------------------------

        if custom == 'CSVfile (name, parallax, parallax_error, ra, dec)':
        
            if model == 'Photogeometric':
                
                print('Photogeometric model only available for modes "single" and "CSVfile(source_id only)"')
                
            else:
                rmeds = []
                rlos = []        
                rhis = []
                
                rmeds_cat = []
                rlos_cat = []
                rhis_cat = []
            
                p = PdfPages('./results/{}.pdf'.format(filename_out))
                
                with open('./data/{}.csv'.format(filename_in), newline='') as f1:
                    reader = csv.reader(f1)
                    rows = []
                    for row in reader:
                        rows.append(row)
                    
                    if all(x in rows[0] for x in ['name', 'parallax', 'parallax_error', 'ra', 'dec']):
                    
                        header = ['name','HEALpixel level 5','w [mas]', 'wsd [mas]', 'wsd/w','glon for HEALpixel [deg]','glat for HEALpixel [deg]','rest [pc]','rlo [pc]','rhi [pc]','rlen [pc]']
                        header_MC = ['name','samples']
                        
                        with open('./results/{}_summary.csv'.format(filename_out), 'w', encoding='UTF8') as f2:
                            writer = csv.writer(f2)
                            # write the header
                            writer.writerow(header)
                            
                            with open('./results/{}_MCMCsamples.csv'.format(filename_out), 'w', encoding='UTF8') as f3:
                                
                                writer3 = csv.writer(f3)
                                writer3.writerow(header_MC)
                            
                                for  i in rows[1:]:
                                    
                                    name = int(i[rows[0].index('name')])
                                    w = float(i[rows[0].index('parallax')])
                                    wsd = float(i[rows[0].index('parallax_error')])
                                    ra = float(i[rows[0].index('ra')])
                                    dec = float(i[rows[0].index('dec')])
                                    HP = HEALPix(nside=2**5, order='nested') #level 5
                                    hp = HP.lonlat_to_healpix(ra*u.deg,dec*u.deg) 
                                    
                                    print('name',name)
                                            
                                    if model == 'EDSD':
                                        rlen = float(rows_prior_summary[hp+1][10])  #EDSDrlen in csv-file
                                    if model == 'GGD':  
                                        rlen = float(rows_prior_summary[hp+1][5]) # GGDrlen in csv-file
                                        
                                    rlen_EDSD = float(rows_prior_summary[hp+1][10])      
                                    alpha = float(rows_prior_summary[hp+1][6])
                                    beta = float(rows_prior_summary[hp+1][7])
                                    glon = float(rows_prior_summary[hp+1][1])
                                    glat = float(rows_prior_summary[hp+1][2])
                                    
                                    rInit = float(mode_post3(w=w*1e-3,wsd=wsd*1e-3,rlen = rlen_EDSD,retall = False)) 
                                    rStep = 0.75*rInit*min(1/3, abs(wsd/w))
                                    Nsamp = int(5e3)
                                    Nburnin = int(5e2)
                                    
                                    # Convert inputs
                                    
                                    
                                    w = 1e-3*w
                                    wsd = 1e-3*wsd
                
                                    if model == 'EDSD':
                
                                        failMessage = np.nan
                                        rQuant0 = quantile_distpost3(w=w,wsd=wsd,rlen=rlen,rInit=rInit,rStep=rStep,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,(1-HDIprob)/2,(1+HDIprob)/2]))
                                        rQuant = rQuant0[0]
                
                                    if model == 'GGD':
                
                                        failMessage = np.nan
                                        rQuant0 = quantile_distpost4(w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta,rStep=rStep,rInit=rInit,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,(1-HDIprob)/2,(1+HDIprob)/2]))
                                        rQuant = rQuant0[0]
        
                                
                                    if rQuant['code'] == 1:
                                        rRes = np.array([rQuant['val'][0],rQuant['val'][1],rQuant['val'][2],2])
                                        
                                    else:
                                        failMessage = rQuant['message']
            
                                    if failMessage == rQuant['message']:
                                        rRes = np.empty((6,))
                                        rRes[:] = np.nan
                                        rRes = np.array([rRes[0],rRes[1],rRes[2],rRes[3],rRes[4],0])    
                
                                    #if rRes[3] == 1:
                                     #   print("HDI: probability contained, #steps to find:",HDIinfo)
                        
                                    if type(failMessage) == str :
                                        print(failMessage)
                                    
                                    if rRes[3] == 0:
                                         print('result_flag = 0.Attempting to plot ...')
                
                                    if np.isnan(rRes[0]) or np.isnan(rRes[1]) or np.isnan(rRes[2]) or np.isnan(rRes[3]) :
                                        rplothi = 5*rlen
                                    else:
                                        rplothi = 2*rRes[2]
                                    
                                    data = [name,hp,1e3*w,1e3*wsd,wsd/w,glon,glat,rRes[0],rRes[1],rRes[2],rlen]
                                    
                                    writer.writerow(data)
                                    
                                    if type(rQuant0[1]) == float:   
                                        writer3.writerow([name,rQuant0[1]])
                                    else:
                                        writer3.writerow([name]+rQuant0[1].tolist())
                                        
                                    ##get distances from gaiaedr3_distance
                                    #
                                    #job2 = Gaia.launch_job("select top 10 *"
                                    #                    "from external.gaiaedr3_distance "
                                    #                    "where source_id={}".format(int(i[0])))
                                    #r2 = job2.get_results()
                                    #
                                    #if len(r2['r_med_geo'])==1 :
                                    #    
                                    #    rmeds.append(rRes[0])
                                    #    rlos.append(rRes[1])
                                    #    rhis.append(rRes[2])
                                    #    
                                    #    r_med_geo = float(r2['r_med_geo'])
                                    #    r_lo_geo = float(r2['r_lo_geo'])
                                    #    r_hi_geo = float(r2['r_hi_geo'])
                                    #
                                    #    rmeds_cat.append(r_med_geo)
                                    #    rlos_cat.append(r_lo_geo)
                                    #    rhis_cat.append(r_hi_geo)
                                    
                                    # Plots 
                                    
                                    samp = rQuant0[1]
                                    
                                    if not np.isnan(samp).all():
                                        
                                        fig,ax = plt.subplots(2,1,figsize=(5,5),gridspec_kw={'height_ratios': [1,2]})
                                        r1 = np.arange(0,len(samp))
                                        ax[0].plot(r1[::50],samp[::50])
                                        ax[0].set_xlabel('samplenumber')
                                        Nplot = 1e3
                                        s = np.arange(1/(2*Nplot),1/Nplot*(Nplot+1),1/Nplot)
                                        rplot = s*(rplothi-rplotlo) + rplotlo
                                        
                                        if model == 'GGD':
                                            
                                            Z = scipy.integrate.quad(ud_distpost4,rlo,rhi,args=(w,wsd,rlen,alpha,beta))[0]
                                            dprior = 1/gamma((beta+1)/alpha)*alpha/(rlen**(beta+1))*rplot**beta*np.exp(-(rplot/rlen)**alpha)
                                            if not (Z == 0 or 1/Z == np.inf):
                                                dpost  = ud_distpost4(r=rplot, w=w, wsd=wsd, rlen=rlen,alpha=alpha,beta=beta)/Z
                                            
                                        if model == 'EDSD':
                
                                            Z = scipy.integrate.quad(ud_distpost3,rlo,rhi,args=(w,wsd,rlen))[0]
                                            dprior = (1/(2*rlen**3))*np.exp(-rplot/rlen)*rplot**2
                                            if not (Z == 0 or 1/Z == np.inf):
                                                dpost  = ud_distpost3(r=rplot, w=w, wsd=wsd, rlen=rlen)/Z
                            
                                        
                                        ax[1].plot(1e-3*rplot,dprior,color='green',label = f'{model} Prior')
                                        ax[1].set_xlim([1e-3*rplotlo,1e-3*rplothi])
                                        #ax[1].set_ylim([0,1.05*max(np.array([dprior,dpost]).flatten())])
                                        ax[0].set_title('{}'.format(name))
                                        ax[1].set_xlabel('distance [kpc]')
                                        if not (Z == 0 or 1/Z == np.inf):
                                            ax[1].plot(1e-3*rplot,dpost,color = 'black',label=f'{model} Posterior')
                                            
                                        ax[1].axvline(1e-3*rRes[0],label ='$r_{med}$(quantile 0.5)')
                                        ax[1].axvline(1e-3*rRes[1], linestyle ='--',label='$r_{lo}$ (quantile 0.159)')
                                        ax[1].axvline(1e-3*rRes[2], linestyle ='--',label='$r_{hi}$(quantile 0.841)')
                                        
                                        #if not np.isnan(np.array([r_med_geo,r_lo_geo,r_hi_geo])).any():
                                        #
                                        #    ax[1].axvline(1e-3*r_med_geo,label ='$r_{med}$ (quantile 0.5) from gaiaedr3_distance',color='cornflowerblue')
                                        #    ax[1].axvline(1e-3*r_lo_geo, linestyle ='--',label='$r_{lo}$ (quantile 0.159) from gaiaedr3_distance',color='cornflowerblue')
                                        #    ax[1].axvline(1e-3*r_hi_geo, linestyle ='--',label='$r_{hi}$ (quantile 0.841) from gaiaedr3_distance',color='cornflowerblue')
                                        
                                        ax[1].plot([], [], ' ', label=f'w [mas] = {1e3*w:.3f}')
                                        ax[1].plot([], [], ' ', label=f'wsd [mas] = {1e3*wsd:.3f}')       
                                        ax[1].plot([], [], ' ', label=f'wsd/w = {wsd/w:.3f}')
                                        
                            
                                        ax[1].legend(fontsize='6')
                                        fig.tight_layout()
                                        p.savefig()
                                        plt.close()
                                
                            p.close()
                    else: 
                        print('csv-input is missing data')
                        
                #Comparison plots  
                
                if all(x in rows[0] for x in ['source_id', 'parallax', 'parallax_error', 'ra', 'dec']):
                    
                    rmeds = np.array(rmeds)
                    rhis = np.array(rhis)
                    rlos = np.array(rlos)
                    rmeds_cat = np.array(rmeds_cat)
                    rhis_cat = np.array(rhis_cat)
                    rlos_cats = np.array(rlos_cat)
                    
                    
                    fig2,ax2 = plt.subplots(3,2,figsize=(7,7))
                    fig2.suptitle('Comparison to published distances in gaiaedr3_distance')
                    ax2[0,0].scatter(rmeds_cat,rmeds)
                    ax2[0,0].set_ylabel('estimated distance $r_{est}$ [pc]')
                    ax2[0,0].set_xlabel('distance from catalogue $r_{cat}$ [pc]')
                    ax2[0,1].scatter(rmeds_cat,rmeds-rmeds_cat)
                    ax2[0,1].set_ylabel('$r_{est}-r_{cat}$ [pc]')
                    ax2[0,1].set_xlabel('$r_{cat}$ [pc]')
        
                    sigma_x = ((rhis-rmeds) + (rmeds-rlos))/2
                    sigma_y = ((rhis_cat-rmeds_cat) + (rmeds_cat-rlos_cat))/2             
                    comp = (rmeds-rmeds_cat)/np.sqrt(sigma_x**2+sigma_y**2)
                    
                    ax2[1,0].scatter(rmeds_cat,comp)
                    ax2[1,0].set_xlabel('$r_{cat}$ [pc]')
                    ax2[1,0].set_ylabel('$\\frac{r_{est}-r_{cat}}{\sqrt{\sigma_{est}^2+\sigma_{cat}^2}}$')
                    
                    ax2[1,1].hist(comp)
                    ax2[1,1].set_xlabel('$\\frac{r_{est}-r_{cat}}{\sqrt{\sigma_{est}^2+\sigma_{cat}^2}}$')
                    
                    ax2[2,0].scatter(rmeds_cat,(rmeds-rmeds_cat)/rmeds_cat)
                    ax2[2,0].set_ylabel('$\\frac{r_{est}-r_{cat}}{r_{cat}}$')
                    ax2[2,0].set_xlabel('$r_{cat}$ [pc]')
                    
                    ax2[2,1].remove()
                    
                    fig2.tight_layout()
                    plt.savefig("./results/{}_comparison-plots.pdf".format(filename_out), format="pdf", bbox_inches="tight")
                    
# Input: CSVfile (only source_id)------------------------------------------------------------------------------------------------------------
        
        if custom == 'CSVfile (source_id only)':
            
            rmeds = []
            rlos = []        
            rhis = []
            
            rmeds_cat = []
            rlos_cat = []
            rhis_cat = []
            
            p = PdfPages('./results/{}.pdf'.format(filename_out))
            
            with open('./data/{}.csv'.format(filename_in), newline='') as f1:
                reader = csv.reader(f1)
                rows = []
                for row in reader:
                    rows.append(row)
                    
                if rows[0] == ['source_id']:
                
                    header = ['source_id','HEALpixel level 5','w [mas]', 'wsd [mas]', 'wsd/w','glon for HEALpixel [deg]','glat for HEALpixel [deg]','rest [pc]','rlo [pc]','rhi [pc]','rlen [pc]']
                    header_MC = ['source_id','samples']
                    
                    with open('./results/{}_summary.csv'.format(filename_out), 'w', encoding='UTF8') as f2:
                        writer = csv.writer(f2)
                        # write the header
                        writer.writerow(header)

                        with open('./results/{}_MCMCsamples.csv'.format(filename_out), 'w', encoding='UTF8') as f3:
                            writer3 = csv.writer(f3)
                            writer3.writerow(header_MC) 
                            
                            for i in rows[1:]:
                                job = Gaia.launch_job("select top 10 "
                                              "source_id, parallax, parallax_error,phot_g_mean_mag, nu_eff_used_in_astrometry, pseudocolour, ecl_lat, astrometric_params_solved,bp_rp "
                                              "from gaiadr3.gaia_source "
                                              "where source_id={}".format(int(i[0])))
            
                                r = job.get_results()
                
                                if len(r) == 0:
                        
                                    print('source_id does not exist!')
                            
                                else:
                            
                
                                    w= float(r['parallax'])
                                    wsd = float(r['parallax_error'])   
                                    phot_g_mean_mag = float(r['phot_g_mean_mag'])
                                    nu_eff_used_in_astrometry = float(r['nu_eff_used_in_astrometry'])
                                    pseudocolour = float(r['pseudocolour'])
                                    ecl_lat = float(r['ecl_lat'])
                                    astrometric_params_solved = float(r['astrometric_params_solved'])
                                    bp_rp = float(r['bp_rp'])
                                    
                                    #Zeropoint:
                                    
                                    if astrometric_params_solved == 31 or astrometric_params_solved == 95:
                                    
                                        if  phot_g_mean_mag == np.nan:
                                            wzp = -0.017
                                        else:
                                            wzp = zpt.get_zpt(phot_g_mean_mag, nu_eff_used_in_astrometry, pseudocolour, ecl_lat, astrometric_params_solved)
                            
                                        #Correct for Zeropoint:
                                
                                        w = w - wzp
                            
                                    source_id = int(i[0])
                                    hp = math.floor(source_id / (2**(35)*4**(12-5)))
                                    
                                    print('source_id',source_id)
                                            
                                    if model == 'EDSD':
                                        rlen = float(rows_prior_summary[hp+1][10])  #EDSDrlen in csv-file
                                    if model == 'GGD':  
                                        rlen = float(rows_prior_summary[hp+1][5]) # GGDrlen in csv-file
                                        
                                    rlen_EDSD = float(rows_prior_summary[hp+1][10])      
                                    alpha = float(rows_prior_summary[hp+1][6])
                                    beta = float(rows_prior_summary[hp+1][7])
                                    glon = float(rows_prior_summary[hp+1][1])
                                    glat = float(rows_prior_summary[hp+1][2])
                                    
                                    rInit = float(mode_post3(w=w*1e-3,wsd=wsd*1e-3,rlen = rlen_EDSD,retall = False)) 
                                    rStep = 0.75*rInit*min(1/3, abs(wsd/w))
                                    Nsamp = int(5e3)
                                    Nburnin = int(5e2)
                                    
                                    # Convert inputs
                                    
                                    
                                    w = 1e-3*w
                                    wsd = 1e-3*wsd
                
                                    if model == 'EDSD':
                
                                        failMessage = np.nan
                                        rQuant0 = quantile_distpost3(w=w,wsd=wsd,rlen=rlen,rInit=rInit,rStep=rStep,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,(1-HDIprob)/2,(1+HDIprob)/2]))
                                        rQuant = rQuant0[0]
                
                                    if model == 'GGD':
                
                                        failMessage = np.nan
                                        rQuant0 = quantile_distpost4(w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta,rStep=rStep,rInit=rInit,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,(1-HDIprob)/2,(1+HDIprob)/2]))
                                        rQuant = rQuant0[0]
                                    
                                    if model == 'Photogeometric':
                                    
                                        failMessage = np.nan
                                        rQuant0 = quantile_distpost5(w=w,wsd=wsd,hp=hp,phot_g_mean_mag=phot_g_mean_mag,bp_rp=bp_rp,pseudocolour=pseudocolour,rInit=rInit,Nsamp=Nsamp,Nburnin=Nburnin,probs=np.array([0.5,0.159,0.841]))
                                        rQuant = rQuant0[0]
                                
                                    if rQuant['code'] == 1:
                                        rRes = np.array([rQuant['val'][0],rQuant['val'][1],rQuant['val'][2],2])
                                        
                                    else:
                                        failMessage = rQuant['message']
            
                                    if failMessage == rQuant['message']:
                                        rRes = np.empty((6,))
                                        rRes[:] = np.nan
                                        rRes = np.array([rRes[0],rRes[1],rRes[2],rRes[3],rRes[4],0])    
                
                                    #if rRes[3] == 1:
                                     #   print("HDI: probability contained, #steps to find:",HDIinfo)
                        
                                    if type(failMessage) == str :
                                        print(failMessage)
                                    
                                    if rRes[3] == 0:
                                         print('result_flag = 0.Attempting to plot ...')
                
                                    if np.isnan(rRes[0]) or np.isnan(rRes[1]) or np.isnan(rRes[2]) or np.isnan(rRes[3]) :
                                        rplothi = 5*rlen
                                    else:
                                        rplothi = 2*rRes[2]
                                    
                                    data = [source_id,hp,1e3*w,1e3*wsd,wsd/w,glon,glat,rRes[0],rRes[1],rRes[2],rlen]
                                    
                                    writer.writerow(data)
                                    
                                    if type(rQuant0[1]) == float:   
                                        writer3.writerow([source_id,rQuant0[1]])
                                    else:
                                        writer3.writerow([source_id]+rQuant0[1].tolist())
                                    
                                    
                                    #get distances from gaiaedr3_distance
                                    
                                    job2 = Gaia.launch_job("select top 10 *"
                                                        "from external.gaiaedr3_distance "
                                                        "where source_id={}".format(int(i[0])))
                                    r2 = job2.get_results()
                                    
                                    if len(r2['r_med_geo'])==1 :
                                        
                                        rmeds.append(rRes[0])
                                        rlos.append(rRes[1])
                                        rhis.append(rRes[2])
                                        
                                        r_med_geo = float(r2['r_med_geo'])
                                        r_lo_geo = float(r2['r_lo_geo'])
                                        r_hi_geo = float(r2['r_hi_geo'])
                                        
                                        r_med_photogeo = float(r2['r_med_photogeo'])
                                        r_lo_photogeo = float(r2['r_lo_photogeo'])
                                        r_hi_photogeo = float(r2['r_hi_photogeo'])
                                        
                                        if model == 'Photogeometric':
                                            
                                            rmeds_cat.append(r_med_photogeo)
                                            rlos_cat.append(r_lo_photogeo)
                                            rhis_cat.append(r_hi_photogeo)
                                            
                                        else:
                                            
                                            rmeds_cat.append(r_med_geo)
                                            rlos_cat.append(r_lo_geo)
                                            rhis_cat.append(r_hi_geo)
                                    
                                    # Plots 
                                    
                                    samp = rQuant0[1]
                                    
                                    if not np.isnan(samp).all():
                                        
                                        fig,ax = plt.subplots(2,1,figsize=(5,5),gridspec_kw={'height_ratios': [1,2]})
                                        r1 = np.arange(0,len(samp))
                                        ax[0].plot(r1[::50],samp[::50])
                                        ax[0].set_xlabel('samplenumber')
                                        Nplot = 1e3
                                        s = np.arange(1/(2*Nplot),1/Nplot*(Nplot+1),1/Nplot)
                                        rplot = s*(rplothi-rplotlo) + rplotlo
                                        
                                        if model == 'GGD':
                                            
                                            Z = scipy.integrate.quad(ud_distpost4,rlo,rhi,args=(w,wsd,rlen,alpha,beta))[0]
                                            dprior = 1/gamma((beta+1)/alpha)*alpha/(rlen**(beta+1))*rplot**beta*np.exp(-(rplot/rlen)**alpha)
                                            if not (Z == 0 or 1/Z == np.inf):
                                                dpost  = ud_distpost4(r=rplot, w=w, wsd=wsd, rlen=rlen,alpha=alpha,beta=beta)/Z
                                            
                                        if model == 'EDSD':
                
                                            Z = scipy.integrate.quad(ud_distpost3,rlo,rhi,args=(w,wsd,rlen))[0]
                                            dprior = (1/(2*rlen**3))*np.exp(-rplot/rlen)*rplot**2
                                            if not (Z == 0 or 1/Z == np.inf):
                                                dpost  = ud_distpost3(r=rplot, w=w, wsd=wsd, rlen=rlen)/Z
                                    
                                        if model == 'Photogeometric':
                                            
                                            dpost = r_ud_distpost5_photogeo(r=rplot,parallax=w,parallax_error=wsd,p=hp,phot_g_mean_mag=phot_g_mean_mag,bp_rp=bp_rp,pseudocolour = pseudocolour)
                                            dpost = dpost/max(dpost)
                                            dprior = r_photogeo_dist_prior(r=rplot,rlen=rlen,beta=beta,alpha=alpha,p=hp,phot_g_mean_mag=phot_g_mean_mag,bp_rp=bp_rp,pseudocolour = pseudocolour)
                                            dprior = dprior/max(dprior)
                                            Z = max(dpost)
                            
                                        
                                        ax[1].plot(1e-3*rplot,dprior,color='green',label = f'{model} Prior')
                                        ax[1].set_xlim([1e-3*rplotlo,1e-3*rplothi])
                                        #ax[1].set_ylim([0,1.05*max(np.array([dprior,dpost]).flatten())])
                                        ax[0].set_title('source_id = {}'.format(i[0]))
                                        ax[1].set_xlabel('distance [kpc]')
                                        if not (Z == 0 or 1/Z == np.inf):
                                            ax[1].plot(1e-3*rplot,dpost,color = 'black',label=f'{model} Posterior')
                                            
                                        ax[1].axvline(1e-3*rRes[0],label ='$r_{med}$(quantile 0.5)')
                                        ax[1].axvline(1e-3*rRes[1], linestyle ='--',label='$r_{lo}$ (quantile 0.159)')
                                        ax[1].axvline(1e-3*rRes[2], linestyle ='--',label='$r_{hi}$(quantile 0.841)')
                                        
                                        if model == 'Photogeometric':
                                            
                                            if not np.isnan(np.array([r_med_photogeo,r_lo_photogeo,r_hi_photogeo])).any():
                                                ax[1].plot([], [], ' ', label= 'Distances from catalogue:')
                                                ax[1].axvline(1e-3*r_med_photogeo,label ='$r_{med_{cat}}$ (quantile 0.5)',color='cornflowerblue')
                                                ax[1].axvline(1e-3*r_lo_photogeo, linestyle ='--',label='$r_{lo_{cat}}$ (quantile 0.159)',color='cornflowerblue')
                                                ax[1].axvline(1e-3*r_hi_photogeo, linestyle ='--',label='$r_{hi_{cat}}$ (quantile 0.841)',color='cornflowerblue')
                                            
                                        else:
                                            
                                            if not np.isnan(np.array([r_med_geo,r_lo_geo,r_hi_geo])).any():
                                                
                                                ax[1].plot([], [], ' ', label= 'Distances from catalogue:')
                                                ax[1].axvline(1e-3*r_med_geo,label ='$r_{med_{cat}}$ (quantile 0.5)',color='cornflowerblue')
                                                ax[1].axvline(1e-3*r_lo_geo, linestyle ='--',label='$r_{lo_{cat}}$ (quantile 0.159)',color='cornflowerblue')
                                                ax[1].axvline(1e-3*r_hi_geo, linestyle ='--',label='$r_{hi_{cat}}$ (quantile 0.841)',color='cornflowerblue')
                                        
                                        ax[1].plot([], [], ' ', label=f'w [mas] = {1e3*w:.3f}')
                                        ax[1].plot([], [], ' ', label=f'wsd [mas] = {1e3*wsd:.3f}')       
                                        ax[1].plot([], [], ' ', label=f'wsd/w = {wsd/w:.3f}')
                                        
                            
                                        ax[1].legend(fontsize='6')
                                        fig.tight_layout()
                                        p.savefig()
                                        plt.close()
                            
                        p.close()  
                else:
                    print('csv-file has to contain only one table with header "source_id"; either it contains more than one table or header is wrong')
            
            #Comparison plots  
            
            if rows[0] == ['source_id']:
                
                rmeds = np.array(rmeds)
                rhis = np.array(rhis)
                rlos = np.array(rlos)
                rmeds_cat = np.array(rmeds_cat)
                rhis_cat = np.array(rhis_cat)
                rlos_cats = np.array(rlos_cat)
                
                
                fig2,ax2 = plt.subplots(3,2,figsize=(7,7))
                fig2.suptitle('Comparison to published distances in gaiaedr3_distance')
                ax2[0,0].scatter(rmeds_cat,rmeds)
                ax2[0,0].set_ylabel('estimated distance $r_{est}$ [pc]')
                ax2[0,0].set_xlabel('distance from catalogue $r_{cat}$ [pc]')
                ax2[0,1].scatter(rmeds_cat,rmeds-rmeds_cat)
                ax2[0,1].set_ylabel('$r_{est}-r_{cat}$ [pc]')
                ax2[0,1].set_xlabel('$r_{cat}$ [pc]')
    
                sigma_x = ((rhis-rmeds) + (rmeds-rlos))/2
                sigma_y = ((rhis_cat-rmeds_cat) + (rmeds_cat-rlos_cat))/2             
                comp = (rmeds-rmeds_cat)/np.sqrt(sigma_x**2+sigma_y**2)
                
                ax2[1,0].scatter(rmeds_cat,comp)
                ax2[1,0].set_xlabel('$r_{cat}$ [pc]')
                ax2[1,0].set_ylabel('$\\frac{r_{est}-r_{cat}}{\sqrt{\sigma_{est}^2+\sigma_{cat}^2}}$')
                
                ax2[1,1].hist(comp)
                ax2[1,1].set_xlabel('$\\frac{r_{est}-r_{cat}}{\sqrt{\sigma_{est}^2+\sigma_{cat}^2}}$')
                
                ax2[2,0].scatter(rmeds_cat,(rmeds-rmeds_cat)/rmeds_cat)
                ax2[2,0].set_ylabel('$\\frac{r_{est}-r_{cat}}{r_{cat}}$')
                ax2[2,0].set_xlabel('$r_{cat}$ [pc]')
                
                ax2[2,1].remove()
                
                fig2.tight_layout()
                plt.savefig("./results/{}_comparison-plots.pdf".format(filename_out), format="pdf", bbox_inches="tight")
                
        if custom == 'Single' or custom == 'Custom':
            
# Input: single source_id -----------------------------------------------------------------------------------------------------

            if custom == 'Single':

                job = Gaia.launch_job("select top 10 "
                                "source_id, parallax, parallax_error,phot_g_mean_mag, nu_eff_used_in_astrometry, pseudocolour, ecl_lat, astrometric_params_solved,bp_rp "
                                "from gaiadr3.gaia_source "
                                "where source_id={}".format(int(source_id)))
    
                r = job.get_results()
        
                if len(r) == 0:
                
                    print('source_id does not exist!') 
                    
                else:
                    
                    w= float(r['parallax'])
                    wsd = float(r['parallax_error'])   
                    phot_g_mean_mag = float(r['phot_g_mean_mag'])
                    nu_eff_used_in_astrometry = float(r['nu_eff_used_in_astrometry'])
                    pseudocolour = float(r['pseudocolour'])
                    ecl_lat = float(r['ecl_lat'])
                    astrometric_params_solved = float(r['astrometric_params_solved'])
                    bp_rp = float(r['bp_rp'])
                    
                    #Zeropoint:
                    if astrometric_params_solved == 31 or astrometric_params_solved == 95:
                        
                        if  phot_g_mean_mag == np.nan:
                            wzp = -0.017
                                        
                        else:
                            wzp = zpt.get_zpt(phot_g_mean_mag, nu_eff_used_in_astrometry, pseudocolour, ecl_lat, astrometric_params_solved)
                                    
                        #Correct for Zeropoint:
                        w = w - wzp
                    
                    
                    
                    source_id = int(r['source_id'])
                    hp = math.floor(source_id / (2**(35)*4**(12-5)) )
                    print('source_id',source_id)
                    print('HEALpixel level 5:',hp)
                    
                    if model == 'EDSD' or model == 'Photogeometric':
                        rlen = float(rows_prior_summary[hp+1][10])  #EDSDrlen in csv-file
                    if model == 'GGD':  
                        rlen = float(rows_prior_summary[hp+1][5]) # GGDrlen in csv-file
                    
                    rlen_EDSD = float(rows_prior_summary[hp+1][10])      
                    alpha = float(rows_prior_summary[hp+1][6])
                    beta = float(rows_prior_summary[hp+1][7])
                    glon = float(rows_prior_summary[hp+1][1])
                    glat = float(rows_prior_summary[hp+1][2])
                    
                    rInit = float(mode_post3(w=1e-3*w,wsd=1e-3*wsd,rlen = rlen_EDSD,retall = False)) 
                    rStep = 0.75*rInit*min(1/3, abs(wsd/w))
                    Nsamp = int(5e3)
                    Nburnin = int(5e2)
                    
                    # Convert inputs        
                    
                    w = 1e-3*w
                    wsd = 1e-3*wsd

                    if astrometric_params_solved == 31 or astrometric_params_solved == 95:
                        wzp = 1e-3*wzp
                        
                    if model == 'EDSD':
                        failMessage = np.nan
                        rQuant0 = quantile_distpost3(w=w,wsd=wsd,rlen=rlen,rInit=rInit,rStep=rStep,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,0.159,0.841])) #(1-HDIprob)/2,(1+HDIprob)/2
                        rQuant = rQuant0[0]
                        
                    if model == 'GGD':
                        failMessage = np.nan
                        rQuant0 = quantile_distpost4(w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta,rStep=rStep,rInit=rInit,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,0.159,0.841]))
                        rQuant = rQuant0[0]
                        
                    if model == 'Photogeometric':
                        
                        failMessage = np.nan
                        rQuant0 = quantile_distpost5(w=w,wsd=wsd,hp=hp,phot_g_mean_mag=phot_g_mean_mag,bp_rp=bp_rp,pseudocolour=pseudocolour,rInit=rInit,Nsamp=Nsamp,Nburnin=Nburnin,probs=np.array([0.5,0.159,0.841]))
                        rQuant = rQuant0[0]
                        if rQuant['code'] == 1:
                            rFlag = rQuant0[2]
                            
                    if rQuant['code'] == 1:
                        rRes = np.array([rQuant['val'][0],rQuant['val'][1],rQuant['val'][2],2])
                        
                    else:
                        
                        failMessage = rQuant['message']
                        
                    if failMessage == rQuant['message']:
                        rRes = np.empty((6,))
                        rRes[:] = np.nan
                        rRes = np.array([rRes[0],rRes[1],rRes[2],rRes[3],rRes[4],0])    
                        
                    #if rRes[3] == 1:
                     #   print("HDI: probability contained, #steps to find:",HDIinfo)
        
                    if type(failMessage) == str :
                        
                        print('')
                        print(failMessage)
                        print ('')
                        
                    if rRes[3] == 0:
                         print('result_flag = 0.Attempting to plot ...')
                            
                    if np.isnan(rRes[0]) or np.isnan(rRes[1]) or np.isnan(rRes[2]) or np.isnan(rRes[3]) :
                        rplothi = 5*rlen
                    else:
                        rplothi = 2*rRes[2]
                        
                    # Print summary statistics
                    
                    if astrometric_params_solved == 31 or astrometric_params_solved == 95:
                        print('w[mas]',1e3*(w+wzp))
                        print('Zeropoint wzp [mas]', 1e3*wzp)
                        print('Zeropointcorrected parallax w[mas]',1e3*w)
                    else:
                        print('w[mas]',1e3*w)
                        print('no zeropoint correction, because astrometric_params_solved is not 31 or 95 ')
                    print('wsd/w', wsd/w)
                    
                    if model == 'GGD' or model == 'Photogeometric':
                        print('alpha',alpha)
                        print('beta',beta)
                    
                    if model == 'Photogeometric':
                        if custom == 'Single':
                            print('G',phot_g_mean_mag)
                            print('colour bp-rp',bp_rp)
                            
                    print('')
                    print(f'glon [deg] for HEALpixel {hp}:',glon)
                    print(f'glat [deg] for HEALpixel {hp}:',glat)
                    print('')
                    print('rest [pc]:',rRes[0])
                    print('rlo [pc]:', rRes[1])
                    print('rhi [pc]:', rRes[2])
                    print('rlen [pc]:', rlen)
                    #print('result_flag', rRes[3]) 
                    print('')
                    print('MCMC Initialization [pc]:', rInit)
                    print('MCMC Stepsize [pc]:',rStep)
                    print('MCMC Number of retained iterations[pc]:',Nsamp)
                    print('MCMC Burnin [pc]:',Nburnin)
                    print('')
                    
                    if model == 'Photogeometric' and rQuant['code'] == 1:
                        print('gmaglim   QGmodel')
                        print(rFlag[0],'               ',rFlag[3])
                    
                    header = ['source_id','HEALpixel level 5','w [mas]', 'wsd [mas]', 'wsd/w','glon [deg]','glat [deg]','rest [pc]','rlo [pc]','rhi [pc]','rlen [pc]','result_flag']
                    header_MC = ['source_id','samples'] 
                    
                    # save summary + MCMC chains in file
                    
                    with open('./results/{}_summary.csv'.format(filename_out), 'w', encoding='UTF8') as f2:
                        
                        header = ['source_id','HEALpixel level 5','w [mas]', 'wsd [mas]', 'wsd/w','glon [deg]','glat [deg]','rest [pc]','rlo [pc]','rhi [pc]','rlen [pc]','result_flag']
                        writer = csv.writer(f2)
                        # write the header
                        writer.writerow(header)
                        data = [source_id,hp,1e3*w,1e3*wsd,wsd/w,glon,glat,rRes[0],rRes[1],rRes[2],rlen,rRes[3]]               
                        writer.writerow(data)      
                        
                    with open('./results/{}_MCMCsamples.csv'.format(filename_out), 'w', encoding='UTF8') as f3:
                        writer3 = csv.writer(f3)
                        writer3.writerow(header_MC)
                        if type(rQuant0[1]) == float:   
                            writer3.writerow([source_id,rQuant0[1]])
                        else:
                            writer3.writerow([source_id]+rQuant0[1].tolist())
                    
                    # Plots 
                    
                    samp = rQuant0[1]
                    
                    if not np.isnan(samp).all():
                    
                        fig,ax = plt.subplots(3,1,figsize=(7,7), gridspec_kw={'height_ratios': [1,1,2]})
                        
                        r1 = np.arange(0,len(samp))
                        
                        ax[1].plot(r1[::50],samp[::50])
                        ax[1].set_xlabel('samplenumber')
                        ax[0].hist(samp,bins=100,density=True,label='MCMC samples')
                        r0 = np.linspace(min(samp),max(samp),num=100)
                        
                        Nplot = 1e3
                        s = np.arange(1/(2*Nplot),1/Nplot*(Nplot+1),1/Nplot)
                        rplot = s*(rplothi-rplotlo) + rplotlo
                        
                        
                        if model == 'EDSD':
                            Z = scipy.integrate.quad(ud_distpost3,rlo,rhi,args=(w,wsd,rlen))[0]
                            if not (Z == 0 or 1/Z == np.inf):
                                ax[0].plot(r0,ud_distpost3(r=r0,w=w,wsd=wsd,rlen=rlen)/Z,label='normalized Posterior')
                                dpost  = ud_distpost3(r=rplot, w=w, wsd=wsd, rlen=rlen)/Z
                            dprior = (1/(2*rlen**3))*np.exp(-rplot/rlen)*rplot**2
                            
                            
                        if model == 'GGD':
                            Z = scipy.integrate.quad(ud_distpost4,rlo,rhi,args=(w,wsd,rlen,alpha,beta))[0]
                            if not (Z == 0 or 1/Z == np.inf):
                                ax[0].plot(r0,ud_distpost4(r=r0,w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta)/Z,label='normalized Posterior')
                                dpost  = ud_distpost4(r=rplot, w=w, wsd=wsd, rlen=rlen,alpha=alpha,beta=beta)/Z
                            dprior = 1/gamma((beta+1)/alpha)*alpha/(rlen**(beta+1))*rplot**beta*np.exp(-(rplot/rlen)**alpha)
                            
                            
                        if model == 'Photogeometric':
                            dpost = r_ud_distpost5_photogeo(r=rplot,parallax=w,parallax_error=wsd,p=hp,phot_g_mean_mag=phot_g_mean_mag,bp_rp=bp_rp,pseudocolour = pseudocolour)
                            dpost = dpost/max(dpost)
                            dprior = r_photogeo_dist_prior(r=rplot,rlen=rlen,beta=beta,alpha=alpha,p=hp,phot_g_mean_mag=phot_g_mean_mag,bp_rp=bp_rp,pseudocolour = pseudocolour)
                            dprior = dprior/max(dprior)
                            Z = max(dpost)
                            
                        ax[0].set_xlabel('distance [pc]')
                        ax[0].legend()
                        ax[2].plot(1e-3*rplot,dprior,color='green',label = f'{model} Prior' )
                        ax[2].set_xlim([1e-3*rplotlo,1e-3*rplothi])
                        ax[2].set_xlabel('distance [kpc]')
                        if not (Z == 0 or 1/Z == np.inf):
                            #ax[2].set_ylim([0,1.05*max(np.array([dprior,dpost]).flatten())])
                            ax[2].plot(1e-3*rplot,dpost,color = 'black',label= f'{model} Posterior')
                            
                        ax[2].axvline(1e-3*rRes[0],label ='$r_{med}$ (quantile 0.5)')
                        ax[2].axvline(1e-3*rRes[1], linestyle ='--',label='$r_{lo}$ (quantile 0.159)')
                        ax[2].axvline(1e-3*rRes[2], linestyle ='--',label='$r_{hi}$ (quantile 0.841)')
                                        
                    #Compare to distances published in gaiaedr3_distance
                        
                        print('')
                        print('Comparison to published distances in gaiaedr3_distance:')
                        print('')
                        job2 = Gaia.launch_job("select top 10 *"
                                            "from external.gaiaedr3_distance " 
                                            "where source_id={}".format(int(source_id)))
                        r2 = job2.get_results()
                        
    
                        r_med_geo = float(r2['r_med_geo'])
                        r_lo_geo = float(r2['r_lo_geo'])
                        r_hi_geo = float(r2['r_hi_geo'])
                        
                        r_med_photogeo = float(r2['r_med_photogeo'])
                        r_lo_photogeo = float(r2['r_lo_photogeo'])
                        r_hi_photogeo = float(r2['r_hi_photogeo'])
                        
                        
                        if model != 'Photogeometric':
                            print('geometric posterior:')
                            print('')
                            print('r_med_geo [pc]:',r_med_geo)
                            print('r_lo_geo [pc]:',r_lo_geo)
                            print('r_hi_geo [pc]:',r_hi_geo)
                            
                            sigma_x = ((rRes[2]-rRes[0]) + (rRes[0]-rRes[1]))/2
                            sigma_y = ((r_hi_geo-r_med_geo) + (r_med_geo-r_lo_geo))/2
                            
                            comp = abs(r_med_geo-rRes[0])/np.sqrt(sigma_x**2+sigma_y**2)
                            
                            print('Deviation:',comp)
                            
                            ax[2].plot([], [], ' ', label='Distances from catalogue:')
                            ax[2].axvline(1e-3*r_med_geo,label ='$r_{med_{cat}}$ (quantile 0.5)',color='cornflowerblue', alpha=0.7)
                            ax[2].axvline(1e-3*r_lo_geo, linestyle ='--',label='$r_{lo_{cat}}$ (quantile 0.159)',color='cornflowerblue', alpha=0.7)
                            ax[2].axvline(1e-3*r_hi_geo, linestyle ='--',label='$r_{hi_{cat}}$ (quantile 0.841) ',color='cornflowerblue', alpha=0.7)
                        else:
                            print('photogeometric posterior:')
                            print('')
                            print('r_med_photogeo [pc]:',r_med_photogeo)
                            print('r_lo_photogeo [pc]:',r_lo_photogeo)
                            print('r_hi_photogeo [pc]:',r_hi_photogeo)
                            
                            sigma_x = ((rRes[2]-rRes[0]) + (rRes[0]-rRes[1]))/2
                            sigma_y = ((r_hi_photogeo-r_med_photogeo) + (r_med_photogeo-r_lo_photogeo))/2
                            
                            comp = abs(r_med_photogeo-rRes[0])/np.sqrt(sigma_x**2+sigma_y**2)
                            
                            print('Deviation:',comp)
                            
                            ax[2].plot([], [],' ' ,label='Distances from catalogue:')
                            ax[2].axvline(1e-3*r_med_photogeo,label ='$r_{med_{cat}}$ (quantile 0.5) ',color='cornflowerblue', alpha=0.7)
                            ax[2].axvline(1e-3*r_lo_photogeo, linestyle ='--',label='$r_{lo{cat}}$ (quantile 0.159) ',color='cornflowerblue', alpha=0.7)
                            ax[2].axvline(1e-3*r_hi_photogeo, linestyle ='--',label='$r_{hi_{cat}}$ (quantile 0.841) ',color='cornflowerblue', alpha=0.7)
                            
                            
                        ax[2].plot([], [], ' ', label=f'w [mas] = {1e3*w:.3f}')
                        ax[2].plot([], [], ' ', label=f'wsd [mas] = {1e3*wsd:.3f}')       
                        ax[2].plot([], [], ' ', label=f'wsd/w = {wsd/w:.3f}')
                        
                        ax[2].legend(fontsize='6')
                        
                        fig.tight_layout()
                        plt.savefig("./results/{}.pdf".format(filename_out), format="pdf", bbox_inches="tight")
                    
# Input: Custom Parameters: ------------------------------------------------------------------------------------------------------      

            if custom == 'Custom': 

                w=w
                wsd=wsd
                rlen = rlen
                alpha = alpha 
                beta = beta
                glon = np.nan # Galactic longitude in degrees (0 to 360)
                glat =  np.nan # Galactic latitude (-90 to +90)
                rInit = metrop_start
                rStep = metrop_step
                Nsamp = metrop_Nsamp
                Nburnin = metrop_Nburnin
                source_id = np.nan
                hp = np.nan
                
                
                # Convert inputs        
                        
                w = 1e-3*w
                wsd = 1e-3*wsd
                
                if model == 'EDSD':
                    failMessage = np.nan
                    rQuant0 = quantile_distpost3(w=w,wsd=wsd,rlen=rlen,rInit=rInit,rStep=rStep,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,0.159,0.841])) #(1-HDIprob)/2,(1+HDIprob)/2
                    rQuant = rQuant0[0]
                    
                if model == 'GGD':
                    failMessage = np.nan
                    rQuant0 = quantile_distpost4(w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta,rStep=rStep,rInit=rInit,Nburnin=Nburnin,Nsamp=Nsamp,probs=np.array([0.5,0.159,0.841]))
                    rQuant = rQuant0[0]
                    
                if model == 'Photogeometric':
                    rQuant0 = ({'code':0,'val':np.nan,'message':'Photogeometric model only available for modes "Single" and "CSVfile(source_id only)"' },np.nan)
                    rQuant = rQuant0[0]
  
                if rQuant['code'] == 1:
                    rRes = np.array([rQuant['val'][0],rQuant['val'][1],rQuant['val'][2],2])
                    
                else:
                    
                    failMessage = rQuant['message']
                    
                if failMessage == rQuant['message']:
                    rRes = np.empty((6,))
                    rRes[:] = np.nan
                    rRes = np.array([rRes[0],rRes[1],rRes[2],rRes[3],rRes[4],0])    
                    
                #if rRes[3] == 1:
                 #   print("HDI: probability contained, #steps to find:",HDIinfo)
    
                if type(failMessage) == str :
                    
                    print('')
                    print(failMessage)
                    print ('')
                    
                if rRes[3] == 0:
                     print('result_flag = 0.Attempting to plot ...')
                        
                if np.isnan(rRes[0]) or np.isnan(rRes[1]) or np.isnan(rRes[2]) or np.isnan(rRes[3]) :
                    rplothi = 5*rlen
                else:
                    rplothi = 2*rRes[2]
                    
                # Print summary statistics

                print('w [mas]',1e3*w)
                print('wsd [mas]',1e3*wsd)
                print('wsd/w', wsd/w)
                
                if model == 'GGD' or model == 'Photogeometric':
                    print('alpha',alpha)
                    print('beta',beta)
                
                if model == 'Photogeometric':
                    if custom == 'Single':
                        print('G',phot_g_mean_mag)
                        print('colour bp-rp',bp_rp)
                        
                print('')
                print(f'glon [deg] for HEALpixel {hp}:',glon)
                print(f'glat [deg] for HEALpixel {hp}:',glat)
                print('')
                print('rest [pc]:',rRes[0])
                print('rlo [pc]:', rRes[1])
                print('rhi [pc]:', rRes[2])
                print('rlen [pc]:', rlen)
                #print('result_flag', rRes[3]) 
                print('')
                print('MCMC Initialization [pc]:', rInit)
                print('MCMC Stepsize [pc]:',rStep)
                print('MCMC Number of retained iterations[pc]:',Nsamp)
                print('MCMC Burnin [pc]:',Nburnin)
                print('')
                
                
                header = ['source_id','HEALpixel level 5','w [mas]', 'wsd [mas]', 'wsd/w','glon [deg]','glat [deg]','rest [pc]','rlo [pc]','rhi [pc]','rlen [pc]','result_flag']
                header_MC = ['source_id','samples'] 
                
                # save summary + MCMC chains in file
                
                with open('./results/{}_summary.csv'.format(filename_out), 'w', encoding='UTF8') as f2:
                    
                    header = ['source_id','HEALpixel level 5','w [mas]', 'wsd [mas]', 'wsd/w','glon [deg]','glat [deg]','rest [pc]','rlo [pc]','rhi [pc]','rlen [pc]','result_flag']
                    writer = csv.writer(f2)
                    # write the header
                    writer.writerow(header)
                    data = [source_id,hp,1e3*w,1e3*wsd,wsd/w,glon,glat,rRes[0],rRes[1],rRes[2],rlen,rRes[3]]               
                    writer.writerow(data)      
                    
                with open('./results/{}_MCMCsamples.csv'.format(filename_out), 'w', encoding='UTF8') as f3:
                    writer3 = csv.writer(f3)
                    writer3.writerow(header_MC)
                    if type(rQuant0[1]) == float:   
                        writer3.writerow([source_id,rQuant0[1]])
                    else:
                        writer3.writerow([source_id]+rQuant0[1].tolist())
                
                # Plots 
                
                samp = rQuant0[1]
                
                if not np.isnan(samp).all():
                
                    fig,ax = plt.subplots(3,1,figsize=(7,7), gridspec_kw={'height_ratios': [1,1,2]})
                    
                    r1 = np.arange(0,len(samp))
                    
                    ax[1].plot(r1[::50],samp[::50])
                    ax[1].set_xlabel('samplenumber')
                    ax[0].hist(samp,bins=100,density=True,label='MCMC samples')
                    r0 = np.linspace(min(samp),max(samp),num=100)
                    
                    Nplot = 1e3
                    s = np.arange(1/(2*Nplot),1/Nplot*(Nplot+1),1/Nplot)
                    rplot = s*(rplothi-rplotlo) + rplotlo
                    
                    
                    if model == 'EDSD':
                        Z = scipy.integrate.quad(ud_distpost3,rlo,rhi,args=(w,wsd,rlen))[0]
                        if not (Z == 0 or 1/Z == np.inf):
                            ax[0].plot(r0,ud_distpost3(r=r0,w=w,wsd=wsd,rlen=rlen)/Z,label='normalized Posterior')
                            dpost  = ud_distpost3(r=rplot, w=w, wsd=wsd, rlen=rlen)/Z
                        dprior = (1/(2*rlen**3))*np.exp(-rplot/rlen)*rplot**2
                        
                        
                    if model == 'GGD':
                        Z = scipy.integrate.quad(ud_distpost4,rlo,rhi,args=(w,wsd,rlen,alpha,beta))[0]
                        if not (Z == 0 or 1/Z == np.inf):
                            ax[0].plot(r0,ud_distpost4(r=r0,w=w,wsd=wsd,rlen=rlen,alpha=alpha,beta=beta)/Z,label='normalized Posterior')
                            dpost  = ud_distpost4(r=rplot, w=w, wsd=wsd, rlen=rlen,alpha=alpha,beta=beta)/Z
                        dprior = 1/gamma((beta+1)/alpha)*alpha/(rlen**(beta+1))*rplot**beta*np.exp(-(rplot/rlen)**alpha)
                        
                        
                    ax[0].set_xlabel('distance [pc]')
                    ax[0].legend()
                    ax[2].plot(1e-3*rplot,dprior,color='green',label = f'{model} Prior' )
                    ax[2].set_xlim([1e-3*rplotlo,1e-3*rplothi])
                    ax[2].set_xlabel('distance [kpc]')
                    if not (Z == 0 or 1/Z == np.inf):
                        #ax[2].set_ylim([0,1.05*max(np.array([dprior,dpost]).flatten())])
                        ax[2].plot(1e-3*rplot,dpost,color = 'black',label= f'{model} Posterior')
                        
                    ax[2].axvline(1e-3*rRes[0],label ='$r_{med}$ (quantile 0.5)')
                    ax[2].axvline(1e-3*rRes[1], linestyle ='--',label='$r_{lo}$ (quantile 0.159)')
                    ax[2].axvline(1e-3*rRes[2], linestyle ='--',label='$r_{hi}$ (quantile 0.841)')
                                    
                        
                    ax[2].plot([], [], ' ', label=f'w [mas] = {1e3*w:.3f}')
                    ax[2].plot([], [], ' ', label=f'wsd [mas] = {1e3*wsd:.3f}')       
                    ax[2].plot([], [], ' ', label=f'wsd/w = {wsd/w:.3f}')
                    
                    ax[2].legend(fontsize='6')
                    
                    fig.tight_layout()
                    plt.savefig("./results/{}.pdf".format(filename_out), format="pdf", bbox_inches="tight")

In [5]:
model = widgets.RadioButtons(
    options=['GGD', 'EDSD','Photogeometric'],
    description='Model:',
    disabled=False
)

display(model)



source_id = widgets.Text(
    value='28147497671505664',
    description='source_id',
    disabled=False
)
display(source_id)

filename_in = widgets.Text(
    value='test',
    description='Input .csv-file',
    disabled=False
)
display(filename_in)

filename_out = widgets.Text(
    value='results',
    description='Output .pdf/.csv-file',
    disabled=False,
    style = {'description_width': 'initial'}
)
display(filename_out)

custom=widgets.Select(
    options=['Custom', 'Single', 'CSVfile (source_id only)','CSVfile (name, parallax, parallax_error, ra, dec)'],
    value='Custom',
    # rows=10,
    description='Mode:',
    layout={'width': 'max-content'},
    disabled=False
)
display(custom)

w0 = widgets.FloatSlider(
    value=1.00,
    min=-2.00,
    max=2.00,
    step=0.01,
    description='Parallax [mas]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'},
    
    
)
w = widgets.FloatText()
widgets.jslink((w0, 'value'), (w, 'value'))

wsd0 = widgets.FloatSlider(
    value=0.2,
    min=0,
    max=2.00,
    step=0.01,
    description='Parallax error [mas]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
    
)
wsd = widgets.FloatText()
widgets.jslink((wsd0, 'value'), (wsd, 'value'))

rlen0 = widgets.FloatSlider(
    value=200,
    min=0,
    max=1000,
    step=0.01,
    description='Length scale [pc]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
)
rlen = widgets.FloatText()
widgets.jslink((rlen0, 'value'), (rlen, 'value'))

alpha0 = widgets.FloatSlider(
    value=1.00,
    min=0,
    max=3,
    step=0.01,
    description='alpha',
    disabled = False ,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
)
alpha = widgets.FloatText()
widgets.jslink((alpha0, 'value'), (alpha, 'value'))

beta0 = widgets.FloatSlider(
    value=2.00,
    min=0,
    max=3.00,
    step=0.01,
    description='beta',
    disabled = False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
)
beta = widgets.FloatText()
widgets.jslink((beta0, 'value'), (beta, 'value'))

metrop_start0 = widgets.FloatSlider(
    value=1000,
    min=0,
    max=2000,
    step=1,
    description='starting value of Metropolis algorithm [pc]:',
    disabled = False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
)
metrop_start = widgets.FloatText()
widgets.jslink((metrop_start0, 'value'), (metrop_start, 'value'))

metrop_step0 = widgets.FloatSlider(
    value=250,
    min=0,
    max=500,
    step=1,
    description='stepsize of Metropolis algorithm [pc]:',
    disabled = False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
)
metrop_step = widgets.FloatText()
widgets.jslink((metrop_step0, 'value'), (metrop_step, 'value'))

metrop_Nsamp0 = widgets.IntSlider(
    value = 5000,
    min=0,
    max=10000,
    step=1,
    description='samplenumber of Metropolis algorithm:',
    disabled = False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
)
metrop_Nsamp = widgets.IntText()
widgets.jslink((metrop_Nsamp0, 'value'), (metrop_Nsamp, 'value'))

metrop_Nburnin0 = widgets.IntSlider(
    value = 500,
    min=0,
    max=1000,
    step=1,
    description='Burnin of Metropolis algorithm:',
    disabled = False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.2f',
    style = {'description_width': 'initial'}
)
metrop_Nburnin = widgets.IntText()
widgets.jslink((metrop_Nburnin0, 'value'), (metrop_Nburnin, 'value'))


display(widgets.HBox([w0,w]))
display(widgets.HBox([wsd0,wsd]))
display(widgets.HBox([rlen0,rlen]))
display(widgets.HBox([metrop_start0,metrop_start]))
display(widgets.HBox([metrop_step0,metrop_step]))
display(widgets.HBox([metrop_Nsamp0,metrop_Nsamp]))
display(widgets.HBox([metrop_Nburnin0,metrop_Nburnin]))

display(widgets.HBox([alpha0,alpha]))
display(widgets.HBox([beta0,beta]))


out = widgets.interactive_output(f,{'custom':custom,'source_id':source_id,'filename_in':filename_in,'filename_out':filename_out,'model':model,'w':w,'wsd':wsd,'rlen':rlen,'alpha':alpha,'beta':beta,'metrop_start':metrop_start,'metrop_step':metrop_step,'metrop_Nsamp':metrop_Nsamp,'metrop_Nburnin':metrop_Nburnin})
out


RadioButtons(description='Model:', options=('GGD', 'EDSD', 'Photogeometric'), value='GGD')

Text(value='28147497671505664', description='source_id')

Text(value='test', description='Input .csv-file')

Text(value='results', description='Output .pdf/.csv-file', style=DescriptionStyle(description_width='initial')…

Select(description='Mode:', layout=Layout(width='max-content'), options=('Custom', 'Single', 'CSVfile (source_…

Output()